In [101]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import calendar
import datetime
from scipy.stats import ttest_ind
%matplotlib inline

In [102]:
import pandas as pd
df = pd.read_csv('Thinkful_Chicago_Speed_Camera_Violations.csv')
#use df = pd.read_csv(infile, parse_dates=['datetime'], date_parser=dateparse) to parse the violation date into day/month/year
df.head()  #default is 5 rows returned



,ADDRESS,CAMERA ID,VIOLATION DATE,VIOLATIONS,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,7122 S SOUTH CHICAGO AVE,CHI184,7/23/2019,196,1182950.681,1857907.892,41.765301,-87.604985,"(41.76530133793055, -87.60498537037378)",32.0,21546.0,67.0,121.0,33.0
1,445 W 127TH,CHI045,7/23/2019,102,1175468.446,1820629.229,41.663174,-87.633520,"(41.663174059325534, -87.63352045241697)",30.0,21861.0,50.0,7.0,43.0
2,536 E MORGAN DR,CHI029,7/23/2019,100,1180984.657,1868164.766,41.793493,-87.611876,"(41.793492645896656, -87.6118759701359)",53.0,22260.0,7.0,401.0,4.0
3,4909 N CICERO AVE,CHI149,7/23/2019,69,1143500.842,1932251.247,41.970135,-87.747724,"(41.970134735816295, -87.7477237971432)",28.0,21869.0,14.0,751.0,38.0
4,8020 W FOREST PRESERVE AV,CHI174,7/23/2019,68,1121895.053,1922705.872,41.944319,-87.827378,"(41.944318833069445, -87.8273782786742)",39.0,22254.0,17.0,413.0,19.0


In [104]:
df_clean = df.drop(columns = ['X COORDINATE', 'Y COORDINATE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'Historical Wards 2003-2015', 'Census Tracts'])  
#I chose to drop the coordinate columns as I wasn't going to use them to overlay on a map of Chicago.  Historical Wards had
#little bearing on my report.
df_clean

,ADDRESS,CAMERA ID,VIOLATION DATE,VIOLATIONS,Zip Codes,Community Areas,Wards
0,7122 S SOUTH CHICAGO AVE,CHI184,7/23/2019,196,21546.0,67.0,33.0
1,445 W 127TH,CHI045,7/23/2019,102,21861.0,50.0,43.0
2,536 E MORGAN DR,CHI029,7/23/2019,100,22260.0,7.0,4.0
3,4909 N CICERO AVE,CHI149,7/23/2019,69,21869.0,14.0,38.0
4,8020 W FOREST PRESERVE AV,CHI174,7/23/2019,68,22254.0,17.0,19.0
5,4831 W LAWRENCE AVE,CHI147,7/23/2019,67,21869.0,15.0,50.0
6,5816 W JACKSON,CHI058,7/23/2019,67,22216.0,26.0,7.0
7,2900 W OGDEN,CHI021,7/23/2019,67,21569.0,30.0,14.0
8,7157 S SOUTH CHICAGO AVE,CHI180,7/23/2019,62,21546.0,67.0,33.0
9,2109 E 87TH ST,CHI145,7/23/2019,58,21202.0,44.0,35.0


In [114]:
date_split = df['VIOLATION DATE'].str.split('/', expand = True) #I want to separate out the violation date into month, date, year
violation_month = date_split[0]
violation_day = date_split[1]
violation_year = date_split[2]
#Now I want to add that violation month column into the dataframe
df_clean.columns

Index(['ADDRESS', 'CAMERA ID', 'VIOLATION DATE', 'MONTH', 'VIOLATION MONTH',
       'VIOLATIONS', 'Zip Codes', 'Community Areas', 'Wards'],
      dtype='object')

In [97]:
for x in violation_month:
    if x == 6 or x == 7 or x == 8:
        "Summer"
    elif x == 9 or x== 10 or x ==11:
        "Fall"
    elif x == 12 or x == 1 or x == 2:
        "Winter"
    elif x == 3 or x == 4 or x == 5:
        "Spring"


In [98]:
#As of July 2019, there are 163 speed cameras in Chicago

In [ ]:
#This dataset reflects the daily volume of violations that have occurred in Children's Safety Zones for each camera. 
#The data reflects violations that occurred from July 1, 2014 until present, minus the most recent 14 days. 
#This data may change due to occasional time lags between the capturing of a potential violation and the processing and determination of a violation. 
#The most recent 14 days are not shown due to revised data being submitted to the City of Chicago. 
#The reported violations are those that have been collected by the camera and radar system and reviewed by two separate City contractors. 
#In some instances, due to the inability the registered owner of the offending vehicle, the violation may not be issued as a citation. 
#However, this dataset contains all violations regardless of whether a citation was issued, which provides an accurate view into the Automated Speed Enforcement Program violations taking place in Children's Safety Zones. 
#More information on the Safety Zone Program can be found here: http://www.cityofchicago.org/city/en/depts/cdot/supp_info/children_s_safetyzoneporgramautomaticspeedenforcement.html. The corresponding dataset for red light camera violations is https://data.cityofchicago.org/id/spqx-js37.

In [ ]:
#What is a children's safety zone?

#Safety zones are designated as a 1/8th of a mile boundary around any Chicago parks or schools. 
#The program uses enhanced signage and automated safety cameras to identify and ticket motorists who are breaking the law by exceeding the speed limits.

In [ ]:
#What is a Community Area in Chicago?
#Chicago is made up of 77 community areas that are often grouped into 9 districts or “sides.” 
#The community areas are well defined and do not overlap. 
#Each community area has one or more neighborhoods in it. The neighborhoods are often not well defined and over lap each other.

In [7]:
#Questions to Answer:

#1.Have the overall violations dropped year over year?  #Good, per MagdalenaThat may be too simple of a question since I can answer that both in Pandas and Excel
##Future research, if # violations decreased, then is it because people drive slower? Over time can you track zip code or wards?

#2. How does violation # depend on the season? Adding the season column through Pandas to the violation data, create the function, transform
###date to season.  Logic "if date dec to march, then set to winter"
### Add a season column
#7/1/2014 to 7/1/2019  use apply to change date to SEASON is_a_string(x):
    # First make sure we're operating on a string, then use our string method.
    #return str(x).isdigit()
    #a. Assume season by date
    #b. Create season column with logic in pandas
    #c. Manipulate in excel

#3. How do the violation volume differ based on the areas or zip or wards of Chicago?
##3b. Which zip codes have the most violations?  Least violations?  Show top 3,5,10..


#Final section - proposal for further research - what did i find interesting that could be expanded on for future??
##property prices to areas of high violations


In [ ]:
#Outline
#Abstract, dataset description
##Questions
###Conclusion
####Proposal